<a href="https://colab.research.google.com/github/jrenatom/tarefa1_PL_A1/blob/main/JRM_TRILHA4_BIG_A3_Classificacaov6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Tarefa de Aprofundamento 3** 
## CASO: Controle de Qualidade de Peças de uma Indústria
---
<br>
<br>





A partir deste notebook você vai criar e avaliar diferentes modelos de classificação para prever a qualidade de peças de uma indústria. 

Em seguida, com base nos seus resultados, responda o **questionário do Moodle**. (**dica**: o questionário é de alternativas e as respontas irão ajudá-lo a verificar os seus resultados ao longo do programa). 

# Basic imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Exploração dos Dados



`pieces` é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

**Objetivo:** Seu objetivo é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos. 



In [2]:
df1 = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces1.csv')
df1.head()

,id,A,B,C,D,Quality,Unit
0,559,4.9,3.1,1.5,0.1,Reject,SP
1,629,4.8,3.4,1.6,0.2,Reject,SP
2,192,6.7,2.5,5.8,1.8,Refurbish,RJ
3,359,7.6,3.0,6.6,2.1,Refurbish,RJ
4,9,4.9,3.1,1.5,0.1,Reject,RJ


## Inspecione os dados

In [3]:
# seu código
df1.describe()


,id,A,B,C,D
count,500.0000,500.000000,491.000000,494.000000,496.000000
mean,323.2080,5.829600,3.076782,3.787449,1.235282
std,187.5734,0.817227,0.434519,1.749994,0.772550
min,0.0000,4.300000,2.000000,1.000000,0.100000
25%,152.7500,5.100000,2.800000,1.600000,0.300000
50%,323.0000,5.850000,3.000000,4.400000,1.300000
75%,486.5000,6.400000,3.400000,5.100000,1.800000
max,648.0000,7.900000,4.400000,6.900000,2.500000


## Preparação dos dados

Aqui você vai preparar o dado para o uso nos modelos. Você fará a seleção de atributos, tratamento de valores nulos, o encode dos dados, normalização e a separação dos dados de treinamento e teste.

### Seleção de Atributos

Elimine atributos que não fazem parte do treinamento.

In [4]:
# seu código

df2 = df1.drop(columns= ['id'] )
df2.head()

,A,B,C,D,Quality,Unit
0,4.9,3.1,1.5,0.1,Reject,SP
1,4.8,3.4,1.6,0.2,Reject,SP
2,6.7,2.5,5.8,1.8,Refurbish,RJ
3,7.6,3.0,6.6,2.1,Refurbish,RJ
4,4.9,3.1,1.5,0.1,Reject,RJ


### Tratamento de Dados Ausentes

Verifique as presença de valores ausentes e, se houverem, subtitua pelo valor médio dos atributos.

In [5]:
# seu código
df2['B'].fillna(value=df2['B'].mean(), inplace=True)
df2['C'].fillna(value=df2['C'].mean(), inplace=True)
df2['D'].fillna(value=df2['D'].mean(), inplace=True)

#df[...] = df[...].fillna( ... )


df2.isnull().sum()

A          0
B          0
C          0
D          0
Quality    0
Unit       0
dtype: int64

### Hot encode

Faça os *Hot Encodes* **necessários** dos dados. Lembre-se de verificar se de fato todos os dados categóricos precisam dessa transformação.

In [6]:
# seu código
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


hot_encode = OneHotEncoder(handle_unknown='ignore') 
hot_encode = hot_encode.fit(df2[['Unit']])

display(hot_encode.categories_)

#df2 = ..

df2.head()

[array(['BH', 'RJ', 'SP'], dtype=object)]

,A,B,C,D,Quality,Unit
0,4.9,3.1,1.5,0.1,Reject,SP
1,4.8,3.4,1.6,0.2,Reject,SP
2,6.7,2.5,5.8,1.8,Refurbish,RJ
3,7.6,3.0,6.6,2.1,Refurbish,RJ
4,4.9,3.1,1.5,0.1,Reject,RJ


In [7]:
df3 = pd.DataFrame(hot_encode.transform(df2[['Unit']] ).toarray(), columns = hot_encode.categories_) 

In [8]:
df3.head()

,BH,RJ,SP
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


In [9]:
df2 = df2.drop(columns='Unit')


In [10]:
df2.head()

,A,B,C,D,Quality
0,4.9,3.1,1.5,0.1,Reject
1,4.8,3.4,1.6,0.2,Reject
2,6.7,2.5,5.8,1.8,Refurbish
3,7.6,3.0,6.6,2.1,Refurbish
4,4.9,3.1,1.5,0.1,Reject


In [11]:
df4 = pd.concat([df2,df3], axis=1)

In [12]:
df4.head()

,A,B,C,D,Quality,"(BH,)","(RJ,)","(SP,)"
0,4.9,3.1,1.5,0.1,Reject,0.0,0.0,1.0
1,4.8,3.4,1.6,0.2,Reject,0.0,0.0,1.0
2,6.7,2.5,5.8,1.8,Refurbish,0.0,1.0,0.0
3,7.6,3.0,6.6,2.1,Refurbish,0.0,1.0,0.0
4,4.9,3.1,1.5,0.1,Reject,0.0,1.0,0.0


In [13]:
df4.columns = ['A', 'B', 'C','D','Quality', 'BH', 'RJ', 'SP']
df4.head()

,A,B,C,D,Quality,BH,RJ,SP
0,4.9,3.1,1.5,0.1,Reject,0.0,0.0,1.0
1,4.8,3.4,1.6,0.2,Reject,0.0,0.0,1.0
2,6.7,2.5,5.8,1.8,Refurbish,0.0,1.0,0.0
3,7.6,3.0,6.6,2.1,Refurbish,0.0,1.0,0.0
4,4.9,3.1,1.5,0.1,Reject,0.0,1.0,0.0


In [14]:
df4.describe()

,A,B,C,D,BH,RJ,SP
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,5.829600,3.076782,3.787449,1.235282,0.302000,0.250000,0.448000
std,0.817227,0.430583,1.739441,0.769447,0.459585,0.433446,0.497787
min,4.300000,2.000000,1.000000,0.100000,0.000000,0.000000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000,0.000000,0.000000
50%,5.850000,3.000000,4.400000,1.300000,0.000000,0.000000,0.000000
75%,6.400000,3.400000,5.100000,1.800000,1.000000,0.250000,1.000000
max,7.900000,4.400000,6.900000,2.500000,1.000000,1.000000,1.000000


### Normalize os dados

Normalize os dados com o `StandardScaler` (melhor) ou a função `scale`.

$$ z_i = \frac{x_i - \bar{x}}{\sigma(x)}$$

E não esqueça de excluir dados não úteis ao treinamento. Depois de normalizar os dados os valores devem apresentar média próxima de 0 e desvio padrão próximo de 1.

In [15]:
# seu código

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(df4.drop(columns='Quality'))

X = scaler.transform(df4.drop(columns='Quality'))

X[0:5]

array([[-1.13864481,  0.05397611, -1.31636544, -1.47692919, -0.65777257,
        -0.57735027,  1.1100193 ],
       [-1.26113242,  0.7514042 , -1.25881814, -1.34683562, -0.65777257,
        -0.57735027,  1.1100193 ],
       [ 1.06613215, -1.34088009,  1.1581686 ,  0.73466146, -0.65777257,
         1.73205081, -0.90088523],
       [ 2.16852062, -0.17849993,  1.61854703,  1.12494216, -0.65777257,
         1.73205081, -0.90088523],
       [-1.13864481,  0.05397611, -1.31636544, -1.47692919, -0.65777257,
         1.73205081, -0.90088523]])

(Moodle) **Q1**. Após todas as transformações a soma dos valores absolutos ( `np.abs()` ) dos atributos referentes ao hot encode dos *BH*, *RJ* e *SP* são respectivamente?

*Empregue esta pergunta como ponto de checagem das suas transformações. Se não encontrar os seus valores dentre as alternativas há provavelmente um erro e você deve revisar as transformações.*

In [16]:
# seu código

df5 = pd.DataFrame(X)
df5.columns = ['A', 'B', 'C','D', 'BH', 'RJ', 'SP']
df5.head()



,A,B,C,D,BH,RJ,SP
0,-1.138645,0.053976,-1.316365,-1.476929,-0.657773,-0.577350,1.110019
1,-1.261132,0.751404,-1.258818,-1.346836,-0.657773,-0.577350,1.110019
2,1.066132,-1.340880,1.158169,0.734661,-0.657773,1.732051,-0.900885
3,2.168521,-0.178500,1.618547,1.124942,-0.657773,1.732051,-0.900885
4,-1.138645,0.053976,-1.316365,-1.476929,-0.657773,1.732051,-0.900885


In [17]:
df5.describe()

,A,B,C,D,BH,RJ,SP
count,5.000000e+02,5.000000e+02,5.000000e+02,5.000000e+02,5.000000e+02,5.000000e+02,5.000000e+02
mean,-1.119105e-16,3.552714e-16,2.131628e-17,-9.947598e-17,-2.842171e-17,-3.197442e-17,8.171241e-17
std,1.001002e+00,1.001002e+00,1.001002e+00,1.001002e+00,1.001002e+00,1.001002e+00,1.001002e+00
min,-1.873570e+00,-2.503260e+00,-1.604102e+00,-1.476929e+00,-6.577726e-01,-5.773503e-01,-9.008852e-01
25%,-8.936696e-01,-6.434520e-01,-1.258818e+00,-1.216742e+00,-6.577726e-01,-5.773503e-01,-9.008852e-01
50%,2.498747e-02,-1.784999e-01,3.525064e-01,8.419362e-02,-6.577726e-01,-5.773503e-01,-9.008852e-01
75%,6.986693e-01,7.514042e-01,7.553375e-01,7.346615e-01,1.520282e+00,0.000000e+00,1.110019e+00
max,2.535983e+00,3.076165e+00,1.791189e+00,1.645316e+00,1.520282e+00,1.732051e+00,1.110019e+00


In [18]:
print('\nSoma BH = ', np.abs( df5['BH'] ).sum() )
print('\nSoma RJ = ', np.abs( df5['RJ'] ).sum() )
print('\nSoma SP = ', np.abs( df5['SP'] ).sum() )


Soma BH =  459.12525524087647

Soma RJ =  433.0127018922194

Soma SP =  497.2886485734416


### Conjuntos de Treinamento e Teste

Separe os dados de treinamento e teste (30%, estratificados pela variável objetivo e com random_state = 1984). Não empregue outros parâmetros não solicitados.


In [19]:
# seu código

from sklearn.model_selection import train_test_split

X = df5
y = df4.Quality

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1984)

print( X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(350, 7) (150, 7) (350,) (150,)


In [20]:
X_train.describe()


,A,B,C,D,BH,RJ,SP
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,-0.001960,-0.008721,0.004016,0.001568,-0.029248,-0.009897,0.035622
std,0.999847,0.991001,1.003958,1.014765,0.988286,0.995643,1.004522
min,-1.873570,-2.503260,-1.604102,-1.476929,-0.657773,-0.577350,-0.900885
25%,-0.893670,-0.643452,-1.258818,-1.314312,-0.657773,-0.577350,-0.900885
50%,0.086231,-0.178500,0.352506,0.149240,-0.657773,-0.577350,-0.900885
75%,0.698669,0.751404,0.755337,0.734661,1.520282,-0.577350,1.110019
max,2.291008,3.076165,1.791189,1.645316,1.520282,1.732051,1.110019


In [21]:
X_test.describe()

,A,B,C,D,BH,RJ,SP
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,0.004573,0.020350,-0.009371,-0.003658,0.068246,0.023094,-0.083117
std,1.007030,1.027032,0.997357,0.971454,1.030185,1.016376,0.991089
min,-1.873570,-2.503260,-1.546555,-1.476929,-0.657773,-0.577350,-0.900885
25%,-0.893670,-0.643452,-1.258818,-1.086648,-0.657773,-0.577350,-0.900885
50%,-0.158744,-0.178500,0.294959,0.084194,-0.657773,-0.577350,-0.900885
75%,0.698669,0.751404,0.755337,0.734661,1.520282,1.732051,1.110019
max,2.535983,3.076165,1.791189,1.645316,1.520282,1.732051,1.110019


# K-Vizinhos mais Próximos

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para encontrar o melhor modelo entre os valores de $k$ de $4$ a $11$ e empregando as métricas euclidiana e manhattan. Verifique o modelo obtidos e o `classification_report` desse modelo.

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [22]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = KNeighborsClassifier()

param_grid = {'n_neighbors': [4, 5, 7, 9, 11], 'metric': ['euclidean', 'manhattan']}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()


KNeighborsClassifier(metric='manhattan', n_neighbors=7)

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.88      0.98      0.93        47
   Refurbish       0.98      0.89      0.93        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.95       150
   macro avg       0.95      0.96      0.95       150
weighted avg       0.96      0.95      0.95       150




In [24]:
#print(clf.cv_results_)

(Moodle) **Q2** Qual o melhor modelo de Knn obtido e qual a sua acuracidade? 

(Moodle) **Q3** Qual classe teve os elementos mais erroneamente classificados? 

# Regressão Logística

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar o modelo de regressão logística. Empregue somente os parâmetros padrão do `sciki-learn` para regressão logística, para isso basta empregar `param_grid = {}`. Verifique os resultados obtidos desse modelo com o `classification_report`. 

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [25]:
from sklearn.linear_model import LogisticRegression
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [26]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


base_estimator = LogisticRegression()

In [27]:
param_grid = {}

In [28]:
clf2 = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf2.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(), param_grid={},
             scoring='accuracy')

In [29]:


print(clf2.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred2 = clf2.predict( X_test )
print(classification_report( y_test, y_pred2 ))
print()

LogisticRegression()

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.92      1.00      0.96        47
   Refurbish       1.00      0.93      0.96        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.97       150
   macro avg       0.97      0.98      0.97       150
weighted avg       0.98      0.97      0.97       150




(Moodle) **Q4** Qual a acuracidade do modelo de regressão logística e qual classe teve mais falsos positivos? 

# Aplicando o melhor Modelo

Empregue o melhor modelo obtido acima para estimar a qualidade das peças das 10 peças do conjunto abaixo.

In [30]:
df_new = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces_new.csv')
df_new 

,id,A,B,C,D,Unit
0,182,5.5,2.6,4.4,1.2,SP
1,345,5.8,2.7,4.1,1.0,SP
2,42,5.1,3.5,1.4,0.2,SP
3,37,6.3,2.5,4.9,1.5,BH
4,61,6.0,3.0,4.8,1.8,RJ


In [31]:
df_new.isnull().sum()

id      0
A       0
B       0
C       0
D       0
Unit    0
dtype: int64

## Preparação dos Dados

Lembre-se, as mesmas operações feitas com os dados de treinamento precisarão ser feitas aqui também. 

### Seleção de Atributos 

In [32]:
# seu código

df_new = df_new.drop(columns= 'id' )

In [33]:
df_new.head()

,A,B,C,D,Unit
0,5.5,2.6,4.4,1.2,SP
1,5.8,2.7,4.1,1.0,SP
2,5.1,3.5,1.4,0.2,SP
3,6.3,2.5,4.9,1.5,BH
4,6.0,3.0,4.8,1.8,RJ


### Hot encode  

In [34]:
# seu código
hot_encode = OneHotEncoder(handle_unknown='ignore') 

hot_encode = hot_encode.fit(df_new[['Unit']])

display(hot_encode.categories_)

df_new = pd.concat( [ df_new.drop(columns='Unit'), pd.DataFrame(hot_encode.transform(df_new[['Unit']]).toarray(), columns = list( hot_encode.categories_[0] )) ], axis=1 ) 
df_new.head()

[array(['BH', 'RJ', 'SP'], dtype=object)]

,A,B,C,D,BH,RJ,SP
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0


### Normalize os dados

In [35]:
# seu código

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_new)

X_new = scaler.transform(df_new)

X_new 

array([[-0.5814019 , -0.72000341,  0.37147964,  0.11058147, -0.5       ,
        -0.5       ,  0.81649658],
       [ 0.14535047, -0.44307902,  0.13930487, -0.25802342, -0.5       ,
        -0.5       ,  0.81649658],
       [-1.55040507,  1.77231608, -1.95026813, -1.73244298, -0.5       ,
        -0.5       ,  0.81649658],
       [ 1.35660443, -0.9969278 ,  0.7584376 ,  0.6634888 ,  2.        ,
        -0.5       , -1.22474487],
       [ 0.62985206,  0.38769414,  0.68104601,  1.21639614, -0.5       ,
         2.        , -1.22474487]])

## Predição

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

base_estimator = LogisticRegression()
param_grid = {}


In [37]:
# seu código


clf3 = GridSearchCV(base_estimator, param_grid, cv=5, scoring= 'accuracy')
clf3.fit(X_train, y_train)

y_pred3 = clf3.predict( X_new )


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [38]:


df_new['Prediction'] = y_pred3

df_new

,A,B,C,D,BH,RJ,SP,Prediction
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0,Accept
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0,Accept
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0,Reject
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0,Refurbish
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0,Refurbish


(Moodle) **Q5** Quais as classes obtidas com o melhor modelo para os cinco novos casos? 

# Apêndice: Árvore de Decisão (RESOLVIDO)

Este é apenas um exemplo empregando o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar um modelo de Árvore de Decisão com diferentes parâmetros. Você irá estudar mais detalhtes do modelo de Árvore de Decisão na próxima trilha. Mas ele é um classificador, assim como o modelo de K-Vizinhos mais Próximos e a Regressão Logística, e o uso desse estimador segue os mesmos moldes desses estimadores. Você pode assim aplicá-lo do mesmo modo. 

In [39]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [40]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [ None, 3 ]}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

DecisionTreeClassifier(criterion='entropy')

Detailed classification report:

              precision    recall  f1-score   support

      Accept       1.00      1.00      1.00        47
   Refurbish       1.00      1.00      1.00        55
      Reject       1.00      1.00      1.00        48

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150


